In [6]:
import pandas_datareader.data as web
import datetime
from decimal import Decimal
from dateutil import relativedelta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, plot_heatmaps, resample_apply
import talib as ta
import importlib
import os
import platform
import psutil

# start = datetime.date(2021,1,1)
start = datetime.datetime.today() + relativedelta.relativedelta(years=-1)
end = datetime.date.today()
#https://www.sbisec.co.jp/ETGate/WPLETmgR001Control?OutSide=on&getFlg=on&burl=search_domestic&cat1=domestic&cat2=none&dir=info&file=domestic_info150818_01.html
# data = web.DataReader('7203.T', 'yahoo', start, end)
# data = web.DataReader('SPYD', 'yahoo', start, end)
# data = web.DataReader('1321.T', 'yahoo', start, end)
# data = web.DataReader('1570.T', 'yahoo', start, end)  #NEXT FUNDS 日経平均レバレッジ・インデックス連動型上場投信
# data = web.DataReader('1357.T', 'yahoo', start, end)  #NEXT FUNDS 日経平均ダブルインバース・インデックス連動型上場投信
# data = web.DataReader('BTC-JPY', 'yahoo', start, end)
# data = web.DataReader('^N225', 'yahoo', start, end)
# data = web.DataReader('1655.T', 'yahoo', start, end)   #iシェアーズ S&P500 米国株 ETF　→　取得されるデータがおかしい？？
# data = web.DataReader('2558.T', 'yahoo', start, end)    #MAXIS 米国株式(S&P500)上場投信
# data = web.DataReader('2631.T', 'yahoo', start, end)    #MAXIS ナスダック100上場投信
data = web.DataReader('2559.T', 'yahoo', start, end)      #MAXIS 全世界株式(オール・カントリー)上場投信
data = data.astype("double")
print(data)

initial = 10_000_000
fname = f"html/plot-"

#position
#0 ... display "only long" result
#1 ... display both "only long" and "long and short" result
position = 1

               High      Low     Open    Close   Volume  Adj Close
Date                                                              
2021-09-15  13360.0  13270.0  13320.0  13310.0   9740.0    13310.0
2021-09-16  13370.0  13270.0  13370.0  13270.0   5095.0    13270.0
2021-09-17  13390.0  13270.0  13270.0  13340.0   7600.0    13340.0
2021-09-21  13150.0  12970.0  13050.0  13020.0  39835.0    13020.0
2021-09-22  13010.0  12840.0  12960.0  13000.0  13138.0    13000.0
...             ...      ...      ...      ...      ...        ...
2022-09-08  14495.0  14430.0  14460.0  14440.0   7167.0    14440.0
2022-09-09  14585.0  14500.0  14540.0  14550.0   4700.0    14550.0
2022-09-12  14725.0  14635.0  14670.0  14710.0   8392.0    14710.0
2022-09-13  14845.0  14800.0  14845.0  14820.0   3573.0    14820.0
2022-09-14  14515.0  14375.0  14430.0  14390.0  12573.0    14390.0

[245 rows x 6 columns]


**ボリンジャーバンドとRSI組み合わせ**

In [10]:
import BacktestingStrategies.Strategy_BBDandRSI as btstBBandRsi
importlib.reload(btstBBandRsi)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstBBandRsi.BBandRSI, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + ".html")

# #最適化
p = psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)

output2=bt.optimize(\
    upper_bound = range(50, 85, 5),
    lower_bound = range(10, 45, 5),
    rsi_window=range(10, 30, 2),\
    maximize='Equity Final [$]',\
    method='grid',\
    max_tries = 1000)
print(output2)
display(output2._trades)
display(output2._strategy)
bt.plot(filename = fname + str(output._strategy) + "_optimize.html") 

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstBBandRsi.BBandRSI_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )

    output = bt.run() # バックテスト実行
    print(output) # 実行結果(データ)
    display(output._trades)
    bt.plot(filename = fname + str(output._strategy) + ".html")

    # #最適化
    p = psutil.Process()
    p.nice(psutil.HIGH_PRIORITY_CLASS)

    output2=bt.optimize(\
        upper_bound = range(50, 85, 5),
        lower_bound = range(10, 45, 5),
        rsi_window=range(10, 30, 2),\
        maximize='Equity Final [$]',\
        method='grid',\
        max_tries = 1000)
    print(output2)
    display(output2._trades)
    display(output2._strategy)
    bt.plot(filename = fname + str(output._strategy) + "_optimize.html")

Start                     2021-09-15 00:00:00
End                       2022-09-14 00:00:00
Duration                    364 days 00:00:00
Exposure Time [%]                   49.795918
Equity Final [$]                9889974.70775
Equity Peak [$]                 10431439.7275
Return [%]                          -1.100253
Buy & Hold Return [%]                  8.1142
Return (Ann.) [%]                    -1.13151
Volatility (Ann.) [%]               15.256388
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -11.087204
Avg. Drawdown [%]                   -4.140205
Max. Drawdown Duration      216 days 00:00:00
Avg. Drawdown Duration       54 days 00:00:00
# Trades                                   17
Win Rate [%]                        29.411765
Best Trade [%]                      10.064839
Worst Trade [%]                     -4.103412
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,715,50,51,13978.85450,13650.0,-235130.96750,-0.023525,2021-11-30,2021-12-01,1 days
1,711,51,53,13717.56750,13645.0,-51595.49250,-0.005290,2021-12-01,2021-12-03,2 days
2,708,53,74,13712.54275,14420.0,500879.73300,0.051592,2021-12-03,2022-01-05,33 days
3,761,86,87,13416.08250,13350.0,-50288.78250,-0.004926,2022-01-24,2022-01-25,1 days
4,757,87,88,13416.08250,13180.0,-178714.45250,-0.017597,2022-01-25,2022-01-26,1 days
5,753,88,89,13245.24100,13285.0,29938.52700,0.003002,2022-01-26,2022-01-27,1 days
6,750,89,90,13350.76075,13250.0,-75570.56250,-0.007547,2022-01-27,2022-01-28,1 days
7,746,90,108,13315.58750,13090.0,-168288.27500,-0.016942,2022-01-28,2022-02-25,28 days
8,742,108,115,13154.79550,12615.0,-400528.26100,-0.041034,2022-02-25,2022-03-08,11 days
9,739,115,116,12677.44425,12630.0,-35061.30075,-0.003742,2022-03-08,2022-03-09,1 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 490 configurations.
  output = _optimize_grid()


Start                     2021-09-15 00:00:00
End                       2022-09-14 00:00:00
Duration                    364 days 00:00:00
Exposure Time [%]                   46.530612
Equity Final [$]               10833253.70125
Equity Peak [$]                10833253.70125
Return [%]                           8.332537
Buy & Hold Return [%]                  8.1142
Return (Ann.) [%]                    8.580547
Volatility (Ann.) [%]               15.863728
Sharpe Ratio                         0.540891
Sortino Ratio                        0.818088
Calmar Ratio                         0.795368
Max. Drawdown [%]                  -10.788152
Avg. Drawdown [%]                   -3.536263
Max. Drawdown Duration      137 days 00:00:00
Avg. Drawdown Duration       31 days 00:00:00
# Trades                                   18
Win Rate [%]                        33.333333
Best Trade [%]                      10.734524
Worst Trade [%]                     -4.103412
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,715,50,51,13978.85450,13650.0,-2.351310e+05,-0.023525,2021-11-30,2021-12-01,1 days
1,711,51,53,13717.56750,13645.0,-5.159549e+04,-0.005290,2021-12-01,2021-12-03,2 days
2,708,53,74,13712.54275,14420.0,5.008797e+05,0.051592,2021-12-03,2022-01-05,33 days
3,761,86,87,13416.08250,13350.0,-5.028878e+04,-0.004926,2022-01-24,2022-01-25,1 days
4,757,87,88,13416.08250,13180.0,-1.787145e+05,-0.017597,2022-01-25,2022-01-26,1 days
5,753,88,89,13245.24100,13285.0,2.993853e+04,0.003002,2022-01-26,2022-01-27,1 days
6,750,89,90,13350.76075,13250.0,-7.557056e+04,-0.007547,2022-01-27,2022-01-28,1 days
7,746,90,108,13315.58750,13090.0,-1.682883e+05,-0.016942,2022-01-28,2022-02-25,28 days
8,742,108,115,13154.79550,12615.0,-4.005283e+05,-0.041034,2022-02-25,2022-03-08,11 days
9,739,115,116,12677.44425,12630.0,-3.506130e+04,-0.003742,2022-03-08,2022-03-09,1 days


...

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-699,34,50,14298.86850,13910.0,2.718191e+05,0.027196,2021-11-05,2021-11-30,25 days
1,734,50,51,13978.85450,13650.0,-2.413792e+05,-0.023525,2021-11-30,2021-12-01,1 days
2,731,51,53,13717.56750,13645.0,-5.304684e+04,-0.005290,2021-12-01,2021-12-03,2 days
3,727,53,74,13712.54275,14420.0,5.143214e+05,0.051592,2021-12-03,2022-01-05,33 days
4,-731,74,86,14348.62100,13350.0,7.299920e+05,0.069597,2022-01-05,2022-01-24,19 days
5,836,86,87,13416.08250,13350.0,-5.524497e+04,-0.004926,2022-01-24,2022-01-25,1 days
6,832,87,88,13416.08250,13180.0,-1.964206e+05,-0.017597,2022-01-25,2022-01-26,1 days
7,828,88,89,13245.24100,13285.0,3.292045e+04,0.003002,2022-01-26,2022-01-27,1 days
8,824,89,90,13350.76075,13250.0,-8.302686e+04,-0.007547,2022-01-27,2022-01-28,1 days
9,820,90,108,13315.58750,13090.0,-1.849817e+05,-0.016942,2022-01-28,2022-02-25,28 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 490 configurations.
  output = _optimize_grid()


Start                     2021-09-15 00:00:00
End                       2022-09-14 00:00:00
Duration                    364 days 00:00:00
Exposure Time [%]                   83.265306
Equity Final [$]                11371242.1095
Equity Peak [$]                11384078.86925
Return [%]                          13.712421
Buy & Hold Return [%]                  8.1142
Return (Ann.) [%]                   14.130683
Volatility (Ann.) [%]               21.054819
Sharpe Ratio                         0.671138
Sortino Ratio                        1.114592
Calmar Ratio                         0.952392
Max. Drawdown [%]                  -14.837041
Avg. Drawdown [%]                   -3.642083
Max. Drawdown Duration      145 days 00:00:00
Avg. Drawdown Duration       28 days 00:00:00
# Trades                                   46
Win Rate [%]                        34.782609
Best Trade [%]                      10.734524
Worst Trade [%]                     -4.103412
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-704,41,45,14189.41300,14240.0,-3.561325e+04,-0.003565,2021-11-16,2021-11-22,6 days
1,-703,45,48,14169.51200,14250.0,-5.658306e+04,-0.005680,2021-11-22,2021-11-26,4 days
2,-698,48,50,14179.46250,13910.0,1.880848e+05,0.019004,2021-11-26,2021-11-30,4 days
3,722,50,51,13978.85450,13650.0,-2.374329e+05,-0.023525,2021-11-30,2021-12-01,1 days
4,718,51,53,13717.56750,13645.0,-5.210347e+04,-0.005290,2021-12-01,2021-12-03,2 days
5,715,53,65,13712.54275,13585.0,-9.119307e+04,-0.009301,2021-12-03,2021-12-21,18 days
6,711,65,74,13652.24575,14420.0,5.458733e+05,0.056236,2021-12-21,2022-01-05,15 days
7,-715,74,84,14348.62100,13630.0,5.138140e+05,0.050083,2022-01-05,2022-01-20,15 days
8,786,84,86,13697.46850,13350.0,-2.731102e+05,-0.025367,2022-01-20,2022-01-24,4 days
9,782,86,87,13416.08250,13350.0,-5.167652e+04,-0.004926,2022-01-24,2022-01-25,1 days


<Strategy BBandRSI_WithShortPosition(upper_bound=65,lower_bound=40,rsi_window=10)>

**MACDとRSIの組み合わせ**

In [8]:
import BacktestingStrategies.Strategy_MACDandRSI as btstMACDandRsi
importlib.reload(btstMACDandRsi)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstMACDandRsi.MACDandRSI, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + ".html")

# #最適化
p = psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)

output2=bt.optimize(\
    MACDshort=range(10, 25, 5),\
    MACDlong=range(10, 75, 10),\
    MACDsignal=range(10, 15, 5),\
    upper_bound = range(50, 85, 5),\
    lower_bound = range(10, 45, 5),\
    rsi_window=range(10, 30, 2),\
    maximize='Equity Final [$]',\
    method='grid',\
    constraint = lambda x: (x.MACDlong > x.MACDshort) & (x.MACDshort > x.MACDsignal),\
    max_tries = 1000)
print(output2)
display(output2._trades)
display(output2._strategy)
bt.plot(filename = fname + str(output._strategy) + "_optimize.html")

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstMACDandRsi.MACDandRSI_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )

    output = bt.run() # バックテスト実行
    print(output) # 実行結果(データ)
    display(output._trades)
    bt.plot(filename = fname + str(output._strategy) + ".html")

    # #最適化
    p = psutil.Process()
    p.nice(psutil.HIGH_PRIORITY_CLASS)

    output2=bt.optimize(\
        MACDshort=range(10, 25, 5),\
        MACDlong=range(10, 75, 10),\
        MACDsignal=range(10, 15, 5),\
        upper_bound = range(50, 85, 5),
        lower_bound = range(10, 45, 5),
        rsi_window=range(10, 30, 2),\
        maximize='Equity Final [$]',\
        method='grid',\
        constraint = lambda x: (x.MACDlong > x.MACDshort) & (x.MACDshort > x.MACDsignal),\
        max_tries = 1000)
    print(output2)
    display(output2._trades)
    display(output2._strategy)
    bt.plot(filename = fname + str(output._strategy) + "_optimize.html")

Start                     2021-09-15 00:00:00
End                       2022-09-14 00:00:00
Duration                    364 days 00:00:00
Exposure Time [%]                         0.0
Equity Final [$]                   10000000.0
Equity Peak [$]                    10000000.0
Return [%]                                0.0
Buy & Hold Return [%]                  8.1142
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     0.0
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 515 configurations.
  output = _optimize_grid()


Start                     2021-09-15 00:00:00
End                       2022-09-14 00:00:00
Duration                    364 days 00:00:00
Exposure Time [%]                         0.0
Equity Final [$]                   10000000.0
Equity Peak [$]                    10000000.0
Return [%]                                0.0
Buy & Hold Return [%]                  8.1142
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     0.0
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              NaN
Max. Drawdown [%]                        -0.0
Avg. Drawdown [%]                         NaN
Max. Drawdown Duration                    NaN
Avg. Drawdown Duration                    NaN
# Trades                                    0
Win Rate [%]                              NaN
Best Trade [%]                            NaN
Worst Trade [%]                           NaN
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration


...

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-714,38,81,13990.40300,14005.0,-10422.25800,-0.001043,2021-11-11,2022-01-17,67 days
1,-716,81,106,13935.67525,13200.0,526743.47900,0.052791,2022-01-17,2022-02-22,36 days
2,-800,106,140,13134.66000,14265.0,-904272.00000,-0.086058,2022-02-22,2022-04-13,50 days
3,-677,140,148,14194.38825,14200.0,-3799.15475,-0.000395,2022-04-13,2022-04-25,12 days
4,-680,148,181,14129.71000,13400.0,496202.80000,0.051644,2022-04-25,2022-06-15,51 days
5,-757,181,225,13333.67000,14545.0,-916976.81000,-0.090847,2022-06-15,2022-08-18,64 days
6,-634,225,227,14473.00225,14500.0,-17116.57350,-0.001865,2022-08-18,2022-08-22,4 days
7,-635,227,230,14428.22500,14350.0,49672.87500,0.005422,2022-08-22,2022-08-25,3 days
8,-645,230,244,14278.96750,14430.0,-97415.96250,-0.010577,2022-08-25,2022-09-14,20 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 494 configurations.
  output = _optimize_grid()


Start                     2021-09-15 00:00:00
End                       2022-09-14 00:00:00
Duration                    364 days 00:00:00
Exposure Time [%]                   66.938776
Equity Final [$]                  9388593.034
Equity Peak [$]                10963751.81425
Return [%]                           -6.11407
Buy & Hold Return [%]                  8.1142
Return (Ann.) [%]                   -6.283152
Volatility (Ann.) [%]               16.113639
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -16.828535
Avg. Drawdown [%]                   -7.093838
Max. Drawdown Duration      190 days 00:00:00
Avg. Drawdown Duration       59 days 00:00:00
# Trades                                    5
Win Rate [%]                             40.0
Best Trade [%]                       5.279079
Worst Trade [%]                     -4.722924
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-717,81,106,13935.67525,13200.0,527479.15425,0.052791,2022-01-17,2022-02-22,36 days
1,-801,106,150,13134.66000,13755.0,-496892.34000,-0.047229,2022-02-22,2022-04-27,64 days
2,-732,150,182,13686.91275,13495.0,140480.13300,0.014022,2022-04-27,2022-06-16,50 days
3,-757,182,235,13428.19975,14060.0,-478272.78925,-0.047050,2022-06-16,2022-09-01,77 days
4,-692,235,244,13990.40300,14430.0,-304201.12400,-0.031421,2022-09-01,2022-09-14,13 days


<Strategy MACDandRSI_WithShortPosition(MACDshort=15,MACDlong=70,MACDsignal=10,upper_bound=65,lower_bound=15,rsi_window=26)>

**MACDとボリンジャーバンドの組み合わせ**

In [9]:
import BacktestingStrategies.Strategy_MACDandBBD as btstMACDandBB
importlib.reload(btstMACDandBB)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstMACDandBB.MACDandBBD, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + ".html")

# #最適化
p = psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)

output2=bt.optimize(\
    MACDshort=range(10, 25, 5),\
    MACDlong=range(10, 75, 10),\
    MACDsignal=range(10, 15, 5),\
    maximize='Equity Final [$]',\
    method='grid',\
    constraint = lambda x: (x.MACDlong > x.MACDshort) & (x.MACDshort > x.MACDsignal),\
    max_tries = 1000)
print(output2)
display(output2._trades)
display(output2._strategy)
bt.plot(filename = fname + str(output._strategy) + "_optimize.html")

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstMACDandBB.MACDandBBD_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )

    output = bt.run() # バックテスト実行
    print(output) # 実行結果(データ)
    display(output._trades)
    bt.plot(filename = fname + str(output._strategy) + ".html")

    # #最適化
    p = psutil.Process()
    p.nice(psutil.HIGH_PRIORITY_CLASS)

    output2=bt.optimize(\
        MACDshort=range(10, 25, 5),\
        MACDlong=range(10, 75, 10),\
        MACDsignal=range(10, 15, 5),\
        maximize='Equity Final [$]',\
        method='grid',\
        constraint = lambda x: (x.MACDlong > x.MACDshort) & (x.MACDshort > x.MACDsignal),\
        max_tries = 1000)
    print(output2)
    display(output2._trades)
    display(output2._strategy)
    bt.plot(filename = fname + str(output._strategy) + "_optimize.html")

Start                     2021-09-15 00:00:00
End                       2022-09-14 00:00:00
Duration                    364 days 00:00:00
Exposure Time [%]                        20.0
Equity Final [$]                9840972.77275
Equity Peak [$]                10061865.58125
Return [%]                          -1.590272
Buy & Hold Return [%]                  8.1142
Return (Ann.) [%]                   -1.635335
Volatility (Ann.) [%]                5.916027
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -4.854548
Avg. Drawdown [%]                   -2.428581
Max. Drawdown Duration      225 days 00:00:00
Avg. Drawdown Duration       85 days 00:00:00
# Trades                                    7
Win Rate [%]                        42.857143
Best Trade [%]                       1.501861
Worst Trade [%]                     -3.356333
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,690,74,81,14491.37900,14005.0,-335601.51000,-0.033563,2022-01-05,2022-01-17,12 days
1,688,126,128,14039.15150,14250.0,145063.76800,0.015019,2022-03-24,2022-03-28,4 days
2,684,128,140,14320.53750,14265.0,-37987.65000,-0.003878,2022-03-28,2022-04-13,16 days
3,696,206,219,14039.15150,14220.0,125870.55600,0.012882,2022-07-21,2022-08-09,19 days
4,692,219,223,14290.38900,14445.0,106990.81200,0.010819,2022-08-09,2022-08-16,7 days
5,689,223,230,14516.50275,14350.0,-114720.39475,-0.011470,2022-08-16,2022-08-25,9 days
6,681,244,244,14501.42850,14430.0,-48642.80850,-0.004926,2022-09-14,2022-09-14,0 days


Start                     2021-09-15 00:00:00
End                       2022-09-14 00:00:00
Duration                    364 days 00:00:00
Exposure Time [%]                   20.408163
Equity Final [$]                9950641.34475
Equity Peak [$]                10061865.58125
Return [%]                          -0.493587
Buy & Hold Return [%]                  8.1142
Return (Ann.) [%]                   -0.507653
Volatility (Ann.) [%]                6.082355
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -4.854548
Avg. Drawdown [%]                   -2.428581
Max. Drawdown Duration      225 days 00:00:00
Avg. Drawdown Duration       85 days 00:00:00
# Trades                                    7
Win Rate [%]                        42.857143
Best Trade [%]                       1.501861
Worst Trade [%]                     -3.356333
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,690,74,81,14491.37900,14005.0,-335601.51000,-0.033563,2022-01-05,2022-01-17,12 days
1,688,126,128,14039.15150,14250.0,145063.76800,0.015019,2022-03-24,2022-03-28,4 days
2,684,128,140,14320.53750,14265.0,-37987.65000,-0.003878,2022-03-28,2022-04-13,16 days
3,696,206,219,14039.15150,14220.0,125870.55600,0.012882,2022-07-21,2022-08-09,19 days
4,692,219,223,14290.38900,14445.0,106990.81200,0.010819,2022-08-09,2022-08-16,7 days
5,689,223,231,14516.50275,14510.0,-4480.39475,-0.000448,2022-08-16,2022-08-26,10 days
6,689,244,244,14501.42850,14430.0,-49214.23650,-0.004926,2022-09-14,2022-09-14,0 days


<Strategy MACDandBBD(MACDshort=15,MACDlong=20,MACDsignal=10)>

Start                     2021-09-15 00:00:00
End                       2022-09-14 00:00:00
Duration                    364 days 00:00:00
Exposure Time [%]                   84.489796
Equity Final [$]                 9314860.6025
Equity Peak [$]                  10328727.742
Return [%]                          -6.851394
Buy & Hold Return [%]                  8.1142
Return (Ann.) [%]                   -7.040092
Volatility (Ann.) [%]               16.535108
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -13.121881
Avg. Drawdown [%]                   -4.948985
Max. Drawdown Duration      268 days 00:00:00
Avg. Drawdown Duration       77 days 00:00:00
# Trades                                   14
Win Rate [%]                        35.714286
Best Trade [%]                       5.279079
Worst Trade [%]                     -6.359814
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-714,38,74,13990.40300,14420.0,-306732.25800,-0.030707,2021-11-11,2022-01-05,55 days
1,668,74,81,14491.37900,14005.0,-324901.17200,-0.033563,2022-01-05,2022-01-17,12 days
2,-672,81,106,13935.67525,13200.0,494373.76800,0.052791,2022-01-17,2022-02-22,36 days
3,-750,106,126,13134.66000,13970.0,-626505.00000,-0.063598,2022-02-22,2022-03-24,30 days
4,657,126,128,14039.15150,14250.0,138527.46450,0.015019,2022-03-24,2022-03-28,4 days
5,654,128,140,14320.53750,14265.0,-36321.52500,-0.003878,2022-03-28,2022-04-13,16 days
6,-657,140,148,14194.38825,14200.0,-3686.91975,-0.000395,2022-04-13,2022-04-25,12 days
7,-660,148,181,14129.71000,13400.0,481608.60000,0.051644,2022-04-25,2022-06-15,51 days
8,-736,181,206,13333.67000,13970.0,-468338.88000,-0.047724,2022-06-15,2022-07-21,36 days
9,665,206,219,14039.15150,14220.0,120264.25250,0.012882,2022-07-21,2022-08-09,19 days


Start                     2021-09-15 00:00:00
End                       2022-09-14 00:00:00
Duration                    364 days 00:00:00
Exposure Time [%]                   69.795918
Equity Final [$]                9657520.85575
Equity Peak [$]                10590854.82025
Return [%]                          -3.424791
Buy & Hold Return [%]                  8.1142
Return (Ann.) [%]                     -3.5209
Volatility (Ann.) [%]               16.113797
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -11.591132
Avg. Drawdown [%]                   -6.686645
Max. Drawdown Duration      190 days 00:00:00
Avg. Drawdown Duration       63 days 00:00:00
# Trades                                   14
Win Rate [%]                        57.142857
Best Trade [%]                       6.391332
Worst Trade [%]                     -7.623576
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,690,74,81,14491.37900,14005.0,-335601.51000,-0.033563,2022-01-05,2022-01-17,12 days
1,-693,81,107,13935.67525,13045.0,617237.94825,0.063913,2022-01-17,2022-02-24,38 days
2,-792,107,126,12980.42725,13970.0,-783741.61800,-0.076236,2022-02-24,2022-03-24,28 days
3,676,126,128,14039.15150,14250.0,142533.58600,0.015019,2022-03-24,2022-03-28,4 days
4,673,128,144,14320.53750,14445.0,83763.26250,0.008691,2022-03-28,2022-04-19,22 days
5,-676,144,149,14373.49725,14275.0,66584.14100,0.006853,2022-04-19,2022-04-26,7 days
6,-689,149,182,14204.33875,13495.0,488734.39875,0.049938,2022-04-26,2022-06-16,51 days
7,-765,182,194,13428.19975,13390.0,29222.80875,0.002845,2022-06-16,2022-07-04,18 days
8,-773,194,206,13323.71950,13970.0,-499574.82650,-0.048506,2022-07-04,2022-07-21,17 days
9,698,206,219,14039.15150,14220.0,126232.25300,0.012882,2022-07-21,2022-08-09,19 days


<Strategy MACDandBBD_WithShortPosition(MACDshort=15,MACDlong=30,MACDsignal=10)>